In [ ]:
import xarray as xr
import pandas as pd
import s3fs
import numpy as np

In [ ]:
# Connexion anonyme au bucket public
fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': 'https://object.files.data.gouv.fr'
    }
)
url = "meteofrance-drias/SocleM-Climat-2025/CPRCM/METROPOLE/ALPX-3/CNRM-ESM2-1/r1i1p1f2/CNRM-AROME46t1/ssp370/day/tasAdjust/version-hackathon-102025/tasAdjust_FR-Metro_CNRM-ESM2-1_ssp370_r1i1p1f2_CNRM-MF_CNRM-AROME46t1_v1-r1_MF-CDFt-ANASTASIA-ALPX-3-1991-2020_day_20700101-20791231.nc"

ds = xr.open_dataset(
    fs.open(url, 'rb'),
    engine='h5netcdf'
)

print(ds)
# Voir les dimensions, variables, métadonnées

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 3652, bnds: 2, y: 450, x: 461, nv4: 4)
Coordinates:
  * time       (time) datetime64[ns] 29kB 2070-01-01T12:00:00 ... 2079-12-31T...
  * y          (y) float64 4kB 300.0 302.5 305.0 ... 1.42e+03 1.422e+03
  * x          (x) float64 4kB 307.5 310.0 312.5 ... 1.455e+03 1.458e+03
    lon        (y, x) float64 2MB ...
    lat        (y, x) float64 2MB ...
Dimensions without coordinates: bnds, nv4
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 58kB ...
    lon_bnds   (y, x, nv4) float64 7MB ...
    lat_bnds   (y, x, nv4) float64 7MB ...
    tasAdjust  (time, y, x) float32 3GB ...
Attributes: (12/69)
    title:                         CNRM-AROME46t1 model output corrected over...
    summary:                       Data were post-processed by the Météo-Fran...
    id:                            hdl:21.14103/2ef3135f-d6b6-49c1-9a25-143d5...
    Conventions:                   CF-1.11 ACDD-1.3
    standard_name_vocabulary:      CF Stan

In [3]:
# Connexion anonyme au bucket public
fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': 'https://object.files.data.gouv.fr'
    }
)
url = "meteofrance-drias/SocleM-Climat-2025/CPRCM/METROPOLE/ALPX-3/CNRM-ESM2-1/r1i1p1f2/CNRM-AROME46t1/ssp370/day/tasAdjust/version-hackathon-102025/tasAdjust_FR-Metro_CNRM-ESM2-1_ssp370_r1i1p1f2_CNRM-MF_CNRM-AROME46t1_v1-r1_MF-CDFt-ANASTASIA-ALPX-3-1991-2020_day_20700101-20791231.nc"

ds = xr.open_dataset(
    fs.open(url, 'rb'),
    engine='h5netcdf'
)

print(ds)
# Voir les dimensions, variables, métadonnées

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 3652, bnds: 2, y: 450, x: 461, nv4: 4)
Coordinates:
  * time       (time) datetime64[ns] 29kB 2070-01-01T12:00:00 ... 2079-12-31T...
  * y          (y) float64 4kB 300.0 302.5 305.0 ... 1.42e+03 1.422e+03
  * x          (x) float64 4kB 307.5 310.0 312.5 ... 1.455e+03 1.458e+03
    lon        (y, x) float64 2MB ...
    lat        (y, x) float64 2MB ...
Dimensions without coordinates: bnds, nv4
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 58kB ...
    lon_bnds   (y, x, nv4) float64 7MB ...
    lat_bnds   (y, x, nv4) float64 7MB ...
    tasAdjust  (time, y, x) float32 3GB ...
Attributes: (12/69)
    title:                         CNRM-AROME46t1 model output corrected over...
    summary:                       Data were post-processed by the Météo-Fran...
    id:                            hdl:21.14103/2ef3135f-d6b6-49c1-9a25-143d5...
    Conventions:                   CF-1.11 ACDD-1.3
    standard_name_vocabulary:      CF Stan

In [19]:
lat, lon = ds.lat.load(), ds.lon.load()
mask = ((lat >= 48.1) & (lat <= 49.1) & (lon >= 1.4) & (lon <= 3.6))

y_idx, x_idx = np.where(mask.values)
y_slice = slice(y_idx.min(), y_idx.max()+1)
x_slice = slice(x_idx.min(), x_idx.max()+1)

# Un jour
ds_jour = ds.sel(time='2075-07-15').isel(y=y_slice, x=x_slice).load()

# Ou une période
# ds_periode = ds.sel(time=slice('2075-07-01', '2075-07-31')).isel(y=y_slice, x=x_slice).load()

df = ds_jour.to_dataframe()

In [20]:
lat, lon = ds.lat.load(), ds.lon.load()
mask = ((lat >= 48.1) & (lat <= 49.1) & (lon >= 1.4) & (lon <= 3.6))  # IDF
 
y_idx, x_idx = np.where(mask.values)
y_slice = slice(y_idx.min(), y_idx.max()+1)
x_slice = slice(x_idx.min(), x_idx.max()+1)
 
# Un jour
ds_jour = ds.sel(time='2075-07-15').isel(y=y_slice, x=x_slice).load()
 
df = ds_jour.to_dataframe().reset_index()
df_cleaned = df[["time", "lon", "lat", "tasAdjust"]]

In [ ]:
df_cleaned

,time,bnds,y,x,nv4,time_bnds,lon_bnds,lat_bnds,tasAdjust,lon,lat
0,2075-07-15 12:00:00,0,1045.0,805.0,0,2075-07-15,1.38710,48.050375,294.964783,1.4032,48.062
1,2075-07-15 12:00:00,0,1045.0,805.0,1,2075-07-15,1.42070,48.051325,294.964783,1.4032,48.062
2,2075-07-15 12:00:00,0,1045.0,805.0,2,2075-07-15,1.41925,48.073775,294.964783,1.4032,48.062
3,2075-07-15 12:00:00,0,1045.0,805.0,3,2075-07-15,1.38565,48.072825,294.964783,1.4032,48.062
4,2075-07-15 12:00:00,0,1045.0,807.5,0,2075-07-15,1.42070,48.051325,294.896240,1.4368,48.063
...,...,...,...,...,...,...,...,...,...,...,...
25187,2075-07-15 12:00:00,1,1160.0,967.5,3,2075-07-16,3.54710,49.148625,291.805298,3.5645,49.138
25188,2075-07-15 12:00:00,1,1160.0,970.0,0,2075-07-16,3.58190,49.126525,291.750610,3.5988,49.138
25189,2075-07-15 12:00:00,1,1160.0,970.0,1,2075-07-16,3.61620,49.126850,291.750610,3.5988,49.138
25190,2075-07-15 12:00:00,1,1160.0,970.0,2,2075-07-16,3.61570,49.149300,291.750610,3.5988,49.138


In [4]:
# Créer un masque pour l'IDF
lat_min, lat_max = 48.1, 49.1
lon_min, lon_max = 1.4, 3.6

mask = (
    (ds.lat >= lat_min) & (ds.lat <= lat_max) &
    (ds.lon >= lon_min) & (ds.lon <= lon_max)
)

# Vérifier combien de points correspondent
print(f"Nombre de points dans l'IDF: {mask.sum().values}")

# Si le masque n'est pas vide, extraire les indices
if mask.sum() > 0:
    y_indices, x_indices = np.where(mask.values)
    y_min_idx, y_max_idx = y_indices.min(), y_indices.max()
    x_min_idx, x_max_idx = x_indices.min(), x_indices.max()
    
    print(f"Indices y: {y_min_idx} - {y_max_idx}")
    print(f"Indices x: {x_min_idx} - {x_max_idx}")
    
    # Extraire la région
    ds_idf = ds.isel(
        y=slice(y_min_idx, y_max_idx+1),
        x=slice(x_min_idx, x_max_idx+1)
    )
    
    print(f"\nDataset IDF: {ds_idf.dims}")
else:
    print("Aucun point trouvé - vérifie les coordonnées lat/lon")

Nombre de points dans l'IDF: 2879
Indices y: 298 - 344
Indices x: 199 - 265

Dataset IDF: FrozenMappingWarningOnValuesAccess({'time': 3652, 'bnds': 2, 'y': 47, 'x': 67, 'nv4': 4})


In [10]:
# Convertir en DataFrame pour analyse
df = ds_idf.to_dask_dataframe()

ImportError: chunk manager 'dask' is not available. Please make sure 'dask' is installed and importable.

In [15]:
df.head()

,time,bnds,y,x,nv4,time_bnds,lon_bnds,lat_bnds,tasAdjust,lon,lat
